In [10]:
import configparser

# Define o caminho do arquivo INI
file_path = "config.ini"

# Define a seção e as chaves desejadas
section = "General"
key = "name"

# Cria o parser e lê o arquivo
config = configparser.ConfigParser()
config.read(file_path)

data = config[section][key]

# data = {key: config[section][key] for key in keys if key in config[section]}
data = config[section][key]
print(data)

John Doe


In [11]:
import os
import pandas as pd
from IPython.display import Image, display

# Change to the parent directory to access the DFVC module
os.chdir('..')
from src import DFVC

# Configure pandas display options for better readability
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns

# Define constant for database access
DB_DIR = 'data'

# Define constants for raw data acess
RAW_HASH = '5cbe11afc7682723929c6216ce499a381aa486127b6997045b7d27e173ca2505'
RAW_DATA_FILENAME = 'raw.dfvc'
RAW_DATA_PATH = os.path.join(DB_DIR, RAW_DATA_FILENAME)

# Define constants for raw data processing (preprocessing step 1)
PRP_S1_HASH = ' '
PRP_S1_FILENAME = 'prp_s1.dfvc'
PRP_S1_PATH = os.path.join(DB_DIR, PRP_S1_FILENAME)

In [2]:
raw_dfvc = DFVC.load_dfvc_file(RAW_DATA_PATH)
raw_dfvc.compare_versions(RAW_HASH)
display(raw_dfvc)

Version integrity verified successfully.


In [ ]:
raw = raw_dfvc.get_dataframe()
raw.head()

In [ ]:
# quarter_label
prp_s1_interim = pd.DataFrame(columns=['quarter', 'quarter_label', 'ig_maus', 'ig_revs', 'tk_maus', 'tk_revs'])
prp_s1_interim['quarter_label'] = raw['timeseries_category'].unique()

In [ ]:
# quarter
def converter_para_data(trimestre):
    tri, ano = trimestre.split()
    tri = int(tri[1])
    if tri == 1:
        mes, dia = 3, 31
    elif tri == 2:
        mes, dia = 6, 30
    elif tri == 3:
        mes, dia = 9, 30
    elif tri == 4:
        mes, dia = 12, 31
    return pd.Timestamp(year=int(ano), month=mes, day=dia)

# Aplicar a transformação ao DataFrame df
prp_s1_interim['quarter'] = prp_s1_interim['quarter_label'].apply(converter_para_data)

prp_s1_interim = prp_s1_interim.sort_values(by='quarter').reset_index(drop=True)

# Definir o ano atual
current_year = pd.Timestamp.now().year

# Filtrar os dados para manter apenas os últimos 5 anos
prp_s1_interim = prp_s1_interim[prp_s1_interim['quarter'].dt.year >= (current_year - 6)]

# Resetar o índice
prp_s1_interim.reset_index(drop=True, inplace=True)

In [ ]:
# Função para preencher o prp_s1_interim com os valores do raw
def preencher_prp_s1_interim(raw, prp_s1_interim):
    for index, row in raw.iterrows():
        if 'Instagram' in row['iframe_title']:
            if 'monthly app users' in row['iframe_title']:
                prp_s1_interim.loc[prp_s1_interim['quarter_label'] == row['timeseries_category'], 'ig_maus'] = row['timeseries_value']
            elif 'revenues' in row['iframe_title']:
                prp_s1_interim.loc[prp_s1_interim['quarter_label'] == row['timeseries_category'], 'ig_revs'] = row['timeseries_value']
        elif 'TikTok' in row['iframe_title']:
            if 'MAUs' in row['iframe_title']:
                prp_s1_interim.loc[prp_s1_interim['quarter_label'] == row['timeseries_category'], 'tk_maus'] = row['timeseries_value']
            elif 'revenues' in row['iframe_title']:
                prp_s1_interim.loc[prp_s1_interim['quarter_label'] == row['timeseries_category'], 'tk_revs'] = row['timeseries_value']
    return prp_s1_interim

# Preencher o prp_s1_interim
prp_s1_interim = preencher_prp_s1_interim(raw, prp_s1_interim)

In [ ]:
# Verifique se as colunas existem no DataFrame
columns_to_convert = ['ig_maus', 'ig_revs', 'tk_maus', 'tk_revs']
for column in columns_to_convert:
    if column in prp_s1_interim.columns:
        # Converta a coluna para int, lidando com possíveis valores nulos
        prp_s1_interim[column] = pd.to_numeric(prp_s1_interim[column], errors='coerce').fillna(0).astype(int)
    else:
        print(f"Coluna {column} não encontrada no DataFrame.")

In [ ]:
prp_s1 = DFVC(prp_s1_interim)
display(prp_s1)
prp_s1.compare_versions(prp_s1_hash)

In [ ]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
prp_s1.export_as_dfvc_file(output_path)